Extraccion transformacion y carga de archivo "output_steam_games"

In [1]:
import pandas as pd
import json
from collections import Counter

In [2]:
# Leer datos de JSON e importar en dataframe
rows = []

with open('output_steam_games.json', encoding='MacRoman') as f:
    for line in f.readlines():
        data = json.loads(line)
        rows.append(data)

df_games = pd.DataFrame(rows)

In [3]:
df_games.head(1)

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
# Información general del DataFrame, como tipos de datos y valores faltantes
df_games.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120445 entries, 0 to 120444
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   publisher     24083 non-null  object
 1   genres        28852 non-null  object
 2   app_name      32133 non-null  object
 3   title         30085 non-null  object
 4   url           32135 non-null  object
 5   release_date  30068 non-null  object
 6   tags          31972 non-null  object
 7   reviews_url   32133 non-null  object
 8   specs         31465 non-null  object
 9   price         30758 non-null  object
 10  early_access  32135 non-null  object
 11  id            32133 non-null  object
 12  developer     28836 non-null  object
dtypes: object(13)
memory usage: 11.9+ MB


In [5]:
# Comprobando si hay valores nulos en el DataFrame
df_games.isnull().sum()

publisher       96362
genres          91593
app_name        88312
title           90360
url             88310
release_date    90377
tags            88473
reviews_url     88312
specs           88980
price           89687
early_access    88310
id              88312
developer       91609
dtype: int64

Con esta funcion descubrimos que todas las columnas tienen datos mixtos

In [6]:
# Definir una función para determinar el tipo de datos de una columna
def column_type(column):
    # Obtener los tipos de datos únicos en la columna
    unique_types = column.apply(type).unique()
    
    # Si hay un solo tipo de datos, devolver ese tipo
    if len(unique_types) == 1:
        return unique_types[0].__name__
    
    # Si hay varios tipos de datos, devolver 'mixed'
    return 'mixed'

# Aplicar la función a todas las columnas del DataFrame
result = df_games.apply(column_type, axis=0)

# Mostrar el resultado
print(result)


publisher       mixed
genres          mixed
app_name        mixed
title           mixed
url             mixed
release_date    mixed
tags            mixed
reviews_url     mixed
specs           mixed
price           mixed
early_access    mixed
id              mixed
developer       mixed
dtype: object


Reemplazamos todos los NaN por 'sin datos'

In [7]:
df_juegos = df_games.dropna(how='all')

Se agrega la columna early_access para entrenar el modelo

In [8]:
df_games['early_access']

0           NaN
1           NaN
2           NaN
3           NaN
4           NaN
          ...  
120440    False
120441    False
120442    False
120443    False
120444     True
Name: early_access, Length: 120445, dtype: object

Eliminar las filas donde 'early_access' es NaN

In [9]:

df_games.dropna(subset=['early_access'], inplace=True)

Convierte 'True' a 1 y 'False' a 0 en la columna 'early_access' y Convierte la columna 'early_access' a tipo de datos int

In [10]:

df_games['early_access'] = df_games['early_access'].map({True: 1, False: 0})

# 
df_games['early_access'] = df_games['early_access'].astype(int)

Reemplazamos todos los valores no ASCII por Unknown

In [11]:
def replace_non_ascii_strings(df, replacement_value="Unknown"):
    # Iterar sobre todas las columnas del DataFrame
    for column_name in df.columns:
        # Reemplazar los valores con caracteres no ASCII por el valor de reemplazo
        df[column_name] = df[column_name].apply(lambda x: replacement_value if isinstance(x, str) and any(ord(c) > 128 for c in x) else x)
    
    return df

# Llamar a la función para reemplazar caracteres no ASCII en todas las columnas
df_juegos = replace_non_ascii_strings(df_juegos, replacement_value="Unknown")


Procedemos averiguar que tipo de datos y que cantidad hay en cada columna

In [12]:
def count_column_types(df):
    # Obtener el tipo de datos de cada columna y contar sus ocurrencias
    type_counts = df.applymap(type).apply(lambda x: {str(k).split("'")[1]: v for k, v in x.value_counts().items()})

    # Transponer el resultado y restablecer el índice
    type_counts = type_counts.T.reset_index()

    # Renombrar las columnas
    type_counts.columns = ['Column', 'Type Counts']

    return type_counts

# Llamar a la función en un DataFrame 'df'
result = count_column_types(df_juegos)

# Mostrar el resultado
print(result)





          Column                     Type Counts
0      publisher   {'str': 24083, 'float': 8052}
1         genres  {'list': 28852, 'float': 3283}
2       app_name      {'str': 32133, 'float': 2}
3          title   {'str': 30085, 'float': 2050}
4            url                  {'str': 32135}
5   release_date   {'str': 30068, 'float': 2067}
6           tags   {'list': 31972, 'float': 163}
7    reviews_url      {'str': 32133, 'float': 2}
8          specs   {'list': 31465, 'float': 670}
9          price   {'float': 30225, 'str': 1910}
10  early_access                 {'bool': 32135}
11            id      {'str': 32133, 'float': 2}
12     developer   {'str': 28836, 'float': 3299}


C:\Users\Daniel\AppData\Local\Temp\ipykernel_232\2283447455.py:3: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  type_counts = df.applymap(type).apply(lambda x: {str(k).split("'")[1]: v for k, v in x.value_counts().items()})


eliminamos los NaN de la columna id y los pasamos a String

In [13]:
# Eliminar NaN de la columna 'id' en df_juegos
df_juegos['id'].fillna('', inplace=True)

# Convertir todos los valores de la columna 'id' a string

df_juegos['id'] = df_juegos['id'].apply(str)



Cambiamos todos los valores flotantes a String dela columna release_date

In [14]:
df_juegos['release_date'] = pd.to_datetime(df_juegos['release_date'], format='%Y-%m-%d', errors='coerce')

Verificamos valores de la columna release_date

In [15]:
# Define una función para verificar si una cadena está en formato de fecha
def is_valid_date(date_str):
    try:
        pd.to_datetime(date_str, format='%Y-%m-%d', errors='raise')
        return True
    except ValueError:
        return False

# Verifica si todos los valores en la columna 'release_date' son fechas válidas
all_dates_are_valid = df_juegos['release_date'].apply(is_valid_date).all()

if all_dates_are_valid:
    print("La columna 'release_date' está en formato de fecha válido.")
else:
    print("La columna 'release_date' no está en formato de fecha válido.")




La columna 'release_date' está en formato de fecha válido.


Verificamos la columna

In [16]:
df_juegos['release_date']

88310    2018-01-04
88311    2018-01-04
88312    2017-07-24
88313    2017-12-07
88314           NaT
            ...    
120440   2018-01-04
120441   2018-01-04
120442   2018-01-04
120443   2017-09-02
120444          NaT
Name: release_date, Length: 32135, dtype: datetime64[ns]

Imprimimos el df para verificar el tipo de datos

In [17]:
def count_column_types(df):
    # Obtener el tipo de datos de cada columna y contar sus ocurrencias
    type_counts = df.applymap(type).apply(lambda x: {str(k).split("'")[1]: v for k, v in x.value_counts().items()})

    # Transponer el resultado y restablecer el índice
    type_counts = type_counts.T.reset_index()

    # Renombrar las columnas
    type_counts.columns = ['Column', 'Type Counts']

    return type_counts

# Llamar a la función en un DataFrame 'df'
result = count_column_types(df_juegos)

# Mostrar el resultado
print(result)


          Column                                        Type Counts
0      publisher                      {'str': 24083, 'float': 8052}
1         genres                     {'list': 28852, 'float': 3283}
2       app_name                         {'str': 32133, 'float': 2}
3          title                      {'str': 30085, 'float': 2050}
4            url                                     {'str': 32135}
5   release_date  {'pandas._libs.tslibs.timestamps.Timestamp': 2...
6           tags                      {'list': 31972, 'float': 163}
7    reviews_url                         {'str': 32133, 'float': 2}
8          specs                      {'list': 31465, 'float': 670}
9          price                      {'float': 30225, 'str': 1910}
10  early_access                                    {'bool': 32135}
11            id                                     {'str': 32135}
12     developer                      {'str': 28836, 'float': 3299}


C:\Users\Daniel\AppData\Local\Temp\ipykernel_232\3020584514.py:3: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  type_counts = df.applymap(type).apply(lambda x: {str(k).split("'")[1]: v for k, v in x.value_counts().items()})


Vemos los string de la columna price

In [18]:
# Filtrar las filas donde 'price' es de tipo cadena (string)
strings_en_price = df_juegos[df_juegos['price'].apply(lambda x: isinstance(x, str))]

# Mostrar los valores únicos de tipo cadena en la columna 'price'
unique_strings_en_price = strings_en_price['price'].unique()

# Mostrar los valores únicos
print(unique_strings_en_price)


['Free To Play' 'Free to Play' 'Free' 'Free Demo' 'Play for Free!'
 'Install Now' 'Play WARMACHINE: Tactics Demo' 'Free Mod' 'Install Theme'
 'Third-party' 'Play Now' 'Unknown' 'Play the Demo' 'Starting at $499.00'
 'Starting at $449.00' 'Free to Try' 'Free Movie' 'Free to Use']


Reemplazamos todos los string por 0

In [20]:
# Reemplazar todos los valores de tipo cadena en la columna 'price' por 0
df_juegos['price'] = df_juegos['price'].apply(lambda x: 0 if isinstance(x, str) else x)


Pasamos la columna price en su totalidad a valores que se puedan sumar

In [21]:
# Convierte la columna 'price' de df_games a tipo float, pero establece 'errors' en 'coerce' para manejar los valores no válidos como NaN
df_juegos['price'] = pd.to_numeric(df_juegos['price'], errors='coerce')

# Ahora puedes usar df como lo necesites


In [22]:
def count_column_types(df):
    # Obtener el tipo de datos de cada columna y contar sus ocurrencias
    type_counts = df.applymap(type).apply(lambda x: {str(k).split("'")[1]: v for k, v in x.value_counts().items()})

    # Transponer el resultado y restablecer el índice
    type_counts = type_counts.T.reset_index()

    # Renombrar las columnas
    type_counts.columns = ['Column', 'Type Counts']

    return type_counts

# Llamar a la función en un DataFrame 'df'
result = count_column_types(df_juegos)

# Mostrar el resultado
print(result)


          Column                                        Type Counts
0      publisher                      {'str': 24083, 'float': 8052}
1         genres                     {'list': 28852, 'float': 3283}
2       app_name                         {'str': 32133, 'float': 2}
3          title                      {'str': 30085, 'float': 2050}
4            url                                     {'str': 32135}
5   release_date  {'pandas._libs.tslibs.timestamps.Timestamp': 2...
6           tags                      {'list': 31972, 'float': 163}
7    reviews_url                         {'str': 32133, 'float': 2}
8          specs                      {'list': 31465, 'float': 670}
9          price                                   {'float': 32135}
10  early_access                                    {'bool': 32135}
11            id                                     {'str': 32135}
12     developer                      {'str': 28836, 'float': 3299}


C:\Users\Daniel\AppData\Local\Temp\ipykernel_232\3020584514.py:3: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  type_counts = df.applymap(type).apply(lambda x: {str(k).split("'")[1]: v for k, v in x.value_counts().items()})


In [23]:
df_juegos['price']

88310     4.99
88311     0.00
88312     0.00
88313     0.99
88314     2.99
          ... 
120440    1.99
120441    4.99
120442    1.99
120443    4.99
120444    4.99
Name: price, Length: 32135, dtype: float64

Reemplazar los valores NaN en la columna 'price' con 0

In [24]:

# Reemplazar los valores NaN en la columna 'price' con 0
df_juegos['price'].fillna(0, inplace=True)



In [25]:
def count_column_types(df):
    # Obtener el tipo de datos de cada columna y contar sus ocurrencias
    type_counts = df.applymap(type).apply(lambda x: {str(k).split("'")[1]: v for k, v in x.value_counts().items()})

    # Transponer el resultado y restablecer el índice
    type_counts = type_counts.T.reset_index()

    # Renombrar las columnas
    type_counts.columns = ['Column', 'Type Counts']

    return type_counts

# Llamar a la función en un DataFrame 'df'
result = count_column_types(df_juegos)

# Mostrar el resultado
print(result)



C:\Users\Daniel\AppData\Local\Temp\ipykernel_232\2838975053.py:3: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  type_counts = df.applymap(type).apply(lambda x: {str(k).split("'")[1]: v for k, v in x.value_counts().items()})


          Column                                        Type Counts
0      publisher                      {'str': 24083, 'float': 8052}
1         genres                     {'list': 28852, 'float': 3283}
2       app_name                         {'str': 32133, 'float': 2}
3          title                      {'str': 30085, 'float': 2050}
4            url                                     {'str': 32135}
5   release_date  {'pandas._libs.tslibs.timestamps.Timestamp': 2...
6           tags                      {'list': 31972, 'float': 163}
7    reviews_url                         {'str': 32133, 'float': 2}
8          specs                      {'list': 31465, 'float': 670}
9          price                                   {'float': 32135}
10  early_access                                    {'bool': 32135}
11            id                                     {'str': 32135}
12     developer                      {'str': 28836, 'float': 3299}


Hacemos lo mismo con la columna Genres

In [26]:
def show_unique_values(df, column_name):
    # Obtener todas las listas de la columna especificada y combinarlas en una lista
    all_values = [value for sublist in df[column_name] if isinstance(sublist, list) for value in sublist]
    
    # Convertir los valores únicos en un conjunto para eliminar duplicados
    unique_values_set = set(all_values)
    
    # Convertir el conjunto nuevamente a una lista
    unique_values_list = list(unique_values_set)
    
    # Devolver el resultado
    return unique_values_list

# Llamar a la función para mostrar los valores únicos de la columna 'genres' del DataFrame 'df'
result = show_unique_values(df_juegos, 'genres')

# Mostrar el resultado
print(result)


['Software Training', 'Audio Production', 'Free to Play', 'Photo Editing', 'Indie', 'Action', 'Strategy', 'Early Access', 'Massively Multiplayer', 'Animation &amp; Modeling', 'Racing', 'Simulation', 'Accounting', 'Education', 'Web Publishing', 'Video Production', 'RPG', 'Casual', 'Utilities', 'Design &amp; Illustration', 'Adventure', 'Sports']


Ahora pasamos todos los valores de genres a String

In [26]:
df_juegos['genres'] = df_juegos['genres'].apply(str)



In [27]:
df_juegos['genres']

88310     ['Action', 'Casual', 'Indie', 'Simulation', 'S...
88311          ['Free to Play', 'Indie', 'RPG', 'Strategy']
88312     ['Casual', 'Free to Play', 'Indie', 'Simulatio...
88313                     ['Action', 'Adventure', 'Casual']
88314                                                   nan
                                ...                        
120440        ['Casual', 'Indie', 'Simulation', 'Strategy']
120441                      ['Casual', 'Indie', 'Strategy']
120442                    ['Indie', 'Racing', 'Simulation']
120443                                  ['Casual', 'Indie']
120444                                                  nan
Name: genres, Length: 32135, dtype: object

Verficamos que no haya listas en genres y que sean de tipo String

In [28]:
def count_column_types(df):
    # Obtener el tipo de datos de cada columna y contar sus ocurrencias
    type_counts = df.applymap(type).apply(lambda x: {str(k).split("'")[1]: v for k, v in x.value_counts().items()})

    # Transponer el resultado y restablecer el índice
    type_counts = type_counts.T.reset_index()

    # Renombrar las columnas
    type_counts.columns = ['Column', 'Type Counts']

    return type_counts

# Llamar a la función en un DataFrame 'df'
result = count_column_types(df_juegos)

# Mostrar el resultado
print(result)


          Column                                        Type Counts
0      publisher                      {'str': 24083, 'float': 8052}
1         genres                                     {'str': 32135}
2       app_name                         {'str': 32133, 'float': 2}
3          title                      {'str': 30085, 'float': 2050}
4            url                                     {'str': 32135}
5   release_date  {'pandas._libs.tslibs.timestamps.Timestamp': 2...
6           tags                      {'list': 31972, 'float': 163}
7    reviews_url                         {'str': 32133, 'float': 2}
8          specs                      {'list': 31465, 'float': 670}
9          price                                   {'float': 32135}
10  early_access                                    {'bool': 32135}
11            id                                     {'str': 32135}
12     developer                      {'str': 28836, 'float': 3299}


C:\Users\Daniel\AppData\Local\Temp\ipykernel_12788\3020584514.py:3: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  type_counts = df.applymap(type).apply(lambda x: {str(k).split("'")[1]: v for k, v in x.value_counts().items()})


Vemos el tipo de datos de la columna developer

In [29]:
# Obtener los valores únicos en la columna 'developer'
valores_developer = df_juegos['developer'].unique()

# Mostrar los valores únicos
print(valores_developer)


['Kotoshiro' 'Secret Level SRL' 'Poolians.com' ... 'VersoVR' 'INGAME'
 'Bidoniera Games']


Vemos si contiene nan

In [30]:
# Verificar si la columna 'developer' contiene valores NaN
contiene_nan = df_juegos['developer'].isnull().any()
print("Contiene NaN:", contiene_nan)


Contiene NaN: True


Reemplazamos esos Nan por sin datos

In [31]:
# Reemplazar los valores NaN en la columna 'developer' por "Sin datos"
df_juegos['developer'].fillna("Sin datos", inplace=True)


Volvemo a verificar q no tenga Nan

In [32]:
# Verificar si la columna 'developer' contiene valores NaN
contiene_nan = df_juegos['developer'].isnull().any()
print("Contiene NaN:", contiene_nan)

Contiene NaN: False


Pasamos los valores float que tenia id a string

In [33]:
# Convertir los valores de tipo float en la columna 'id' a cadenas (strings)
df_juegos['id'] = df_juegos['id'].astype(str)

Verificamos que solo sea de numeros la columna id

In [34]:
import pandas as pd
import re

# Crear una función para verificar si una cadena contiene solo letras
def contiene_letras(cadena):
    return bool(re.search('[a-zA-Z]', cadena))

# Verificar si hay valores en la columna 'id' que contienen solo letras
valores_con_letras = df_juegos['id'].apply(lambda x: contiene_letras(x) if isinstance(x, str) and 'id' in x else False)

# Mostrar los valores que contienen letras en la columna 'id'
valores_letras_id = df_juegos[valores_con_letras]

# Mostrar los resultados
print("Valores en la columna 'id' que contienen letras:")
print(valores_letras_id)



Valores en la columna 'id' que contienen letras:
Empty DataFrame
Columns: [publisher, genres, app_name, title, url, release_date, tags, reviews_url, specs, price, early_access, id, developer]
Index: []


Sacamos los valores de listas

In [35]:
# Reemplazar los corchetes y comillas simples y dividir los géneros por comas
df_juegos['genres'] = df_juegos['genres'].apply(lambda x: ', '.join(x.strip("[]").replace("'", "").split(', ')))

# Verificar los primeros registros para asegurarse de que se formateó correctamente
print(df_juegos['genres'].head())


88310        Action, Casual, Indie, Simulation, Strategy
88311                 Free to Play, Indie, RPG, Strategy
88312    Casual, Free to Play, Indie, Simulation, Sports
88313                          Action, Adventure, Casual
88314                                                nan
Name: genres, dtype: object


In [36]:
df_juegos['id'] = df_juegos['id'].apply(lambda x: ', '.join(x.strip("[]").replace("'", "").split(', ')))


In [37]:
df_juegos['tags']

88310         [Strategy, Action, Indie, Casual, Simulation]
88311     [Free to Play, Strategy, Indie, RPG, Card Game...
88312     [Free to Play, Simulation, Sports, Casual, Ind...
88313                           [Action, Adventure, Casual]
88314                       [Action, Indie, Casual, Sports]
                                ...                        
120440                [Strategy, Indie, Casual, Simulation]
120441                            [Strategy, Indie, Casual]
120442                          [Indie, Simulation, Racing]
120443    [Indie, Casual, Puzzle, Singleplayer, Atmosphe...
120444    [Early Access, Adventure, Indie, Action, Simul...
Name: tags, Length: 32135, dtype: object

Se limpia la columna tags para el modelo machine learning

In [38]:
df_juegos['tags'] = df_juegos['tags'].fillna('').apply(lambda x: ', '.join(map(str, x)))

# Verificar los primeros registros para asegurarse de que se formateó correctamente
print(df_juegos['tags'].head())

88310          Strategy, Action, Indie, Casual, Simulation
88311    Free to Play, Strategy, Indie, RPG, Card Game,...
88312    Free to Play, Simulation, Sports, Casual, Indi...
88313                            Action, Adventure, Casual
88314                        Action, Indie, Casual, Sports
Name: tags, dtype: object


Se limpia la columna specs para el modelo machine learning

In [39]:
df_juegos['specs'] = df_juegos['specs'].fillna('').apply(lambda x: ', '.join(map(str, x)))

# Verificar los primeros registros para asegurarse de que se formateó correctamente
print(df_juegos['specs'].head())

88310                                        Single-player
88311    Single-player, Multi-player, Online Multi-Play...
88312    Single-player, Multi-player, Online Multi-Play...
88313                                        Single-player
88314    Single-player, Full controller support, HTC Vi...
Name: specs, dtype: object


Creamos una copia del df con las columnas limpias

In [41]:
# Crear una copia del DataFrame con las columnas específicas
df_games_copia = df_juegos[['genres', 'release_date', 'price', 'developer', 'id']].copy()

Se crea otro df para el modelo

In [42]:

columnas_deseadas = ['genres', 'tags', 'specs']
df_games_model = df_juegos[columnas_deseadas].copy()


Eliminamos las filas que no tengan datos entre las 5 columnas limpias

In [43]:

# Convertir la columna 'price' a tipo numérico
df_games_copia['price'] = pd.to_numeric(df_games_copia['price'], errors='coerce')

# Eliminar las filas que cumplan con las condiciones especificadas
df_games_copia = df_games_copia[~((df_games_copia['developer'] == 'Sin datos') |
                                  (df_games_copia['release_date'].isna()) |
                                  (df_games_copia['genres'].isna()))]



In [44]:
df_games_copia

,genres,release_date,price,developer,id
88310,"Action, Casual, Indie, Simulation, Strategy",2018-01-04,4.99,Kotoshiro,761140
88311,"Free to Play, Indie, RPG, Strategy",2018-01-04,0.00,Secret Level SRL,643980
88312,"Casual, Free to Play, Indie, Simulation, Sports",2017-07-24,0.00,Poolians.com,670290
88313,"Action, Adventure, Casual",2017-12-07,0.99,Unknown,767400
88315,"Action, Adventure, Simulation",2018-01-04,3.99,Trickjump Games Ltd,772540
...,...,...,...,...,...
120439,"Action, Adventure, Casual, Indie",2018-01-04,1.99,Bidoniera Games,745400
120440,"Casual, Indie, Simulation, Strategy",2018-01-04,1.99,"Nikita ""Ghost_RUS""",773640
120441,"Casual, Indie, Strategy",2018-01-04,4.99,Sacada,733530
120442,"Indie, Racing, Simulation",2018-01-04,1.99,Laush Dmitriy Sergeevich,610660


In [45]:
df_games_copia.info()

<class 'pandas.core.frame.DataFrame'>
Index: 28533 entries, 88310 to 120443
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   genres        28533 non-null  object        
 1   release_date  28533 non-null  datetime64[ns]
 2   price         28533 non-null  float64       
 3   developer     28533 non-null  object        
 4   id            28533 non-null  object        
dtypes: datetime64[ns](1), float64(1), object(3)
memory usage: 1.3+ MB


Terminamos la limpieza

Lo guardamos como archivo parques

In [47]:
df_games_copia.to_pickle('games.pkl')

In [50]:
df_games_model.info()

<class 'pandas.core.frame.DataFrame'>
Index: 32135 entries, 88310 to 120444
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   genres  32135 non-null  object
 1   tags    32135 non-null  object
 2   specs   32135 non-null  object
dtypes: object(3)
memory usage: 1004.2+ KB


# Probamos las funciones

In [41]:
import pandas as pd
# Importar el archivo 'reviews.pkl' en formato Pickle
df_reviews = pd.read_pickle('reviews.pkl')

# Importar el archivo 'items.pkl' en formato Pickle
df_items = pd.read_pickle('playtime.pkl')

# Importar el archivo 'games.pkl' en formato Pickle
df_games = pd.read_pickle('games.pkl')




Probamos userdata

In [42]:
def userdata(user_id, df_reviews, df_games):
    # Filtrar las revisiones del usuario en particular
    user_reviews = df_reviews[df_reviews['user_id'] == user_id]

    # Calcular la cantidad de dinero gastado por el usuario
    money_spent = user_reviews['item_id'].apply(lambda x: df_games[df_games['id'] == x]['price'].values[0] if x in df_games['id'].values else 0).sum()

    # Filtrar las revisiones del mismo "item_id"
    item_reviews = df_reviews[df_reviews['item_id'].isin(user_reviews['item_id'])]

    # Calcular el porcentaje de recomendación en base a las revisiones de todos los usuarios del mismo "item_id"
    total_reviews = len(item_reviews)
    positive_reviews = (item_reviews['recommend'] == 'True').sum()
    
    if total_reviews > 0:
        recommendation_percentage = (positive_reviews / total_reviews) * 100
    else:
        recommendation_percentage = 0

    # Obtener la cantidad de elementos revisados
    total_items_reviewed = total_reviews

    return money_spent, recommendation_percentage, total_items_reviewed



In [43]:
user_id = 'js41637'  # Reemplaza con un ID de usuario válido
money_spent, recommendation_percentage, total_items_reviewed = userdata(user_id, df_reviews, df_games)

# Imprimir los resultados
print(f"Usuario {user_id}:")
print(f"Dinero gastado: ${money_spent:.2f}")
print(f"Porcentaje de recomendación: {recommendation_percentage:.2f}%")
print(f"Total de elementos revisados: {total_items_reviewed}")

Usuario js41637:
Dinero gastado: $29.98
Porcentaje de recomendación: 0.00%
Total de elementos revisados: 195


In [44]:
# Filtrar las revisiones que tienen recomendaciones (recommend = True)
users_with_recommendations = df_reviews[df_reviews['recommend'] == True]['user_id'].unique()

# Mostrar algunos de los usuarios que tienen recomendaciones
print("Usuarios con recomendaciones en sus revisiones:")
print(users_with_recommendations[:10])  # Mostrar los primeros 10 usuarios con recomendaciones


Usuarios con recomendaciones en sus revisiones:
['76561197970982479' 'js41637' 'evcentric' 'doctr' 'maplemage' 'Wackky'
 '76561198079601835' 'MeaTCompany' '76561198089393905' '76561198156664158']


Seleccionamos uno de estos usuarios al azar y calculamos su porcentaje de recomendación:

In [45]:
import random

# Seleccionar un usuario al azar de la lista de usuarios con recomendaciones
random_user_with_recommendations = random.choice(users_with_recommendations)

# Mostrar el ID del usuario seleccionado al azar
print(f"Usuario seleccionado al azar con recomendaciones: {random_user_with_recommendations}")

# Calcular el porcentaje de recomendación para el usuario seleccionado
money_spent, recommendation_percentage, total_items_reviewed = userdata(random_user_with_recommendations, df_reviews, df_games)

# Imprimir los resultados
print(f"Usuario {random_user_with_recommendations}:")
print(f"Dinero gastado: ${money_spent:.2f}")
print(f"Porcentaje de recomendación: {recommendation_percentage:.2f}%")
print(f"Total de elementos revisados: {total_items_reviewed}")


Usuario seleccionado al azar con recomendaciones: 76561198058782320
Usuario 76561198058782320:
Dinero gastado: $14.99
Porcentaje de recomendación: 0.00%
Total de elementos revisados: 41


Segunda funcion

In [46]:
def count_reviews_between_dates(df, start_date, end_date):
    # Convierte las fechas dadas a objetos datetime
    start_date = pd.to_datetime(start_date)
    end_date = pd.to_datetime(end_date)

    # Filtra las reseñas que están dentro del rango de fechas
    filtered_reviews = df[(df['posted'] >= start_date) & (df['posted'] <= end_date)]

    # Calcula la cantidad de usuarios que realizaron reseñas
    total_users = len(filtered_reviews['user_id'].unique())

    # Calcula el porcentaje de recomendación
    recommend_percentage = (filtered_reviews['recommend'].astype(int).sum() / len(filtered_reviews)) * 100

    return total_users, recommend_percentage

# Ejemplo de uso
# Reemplaza 'df_reviews' con tu DataFrame de reseñas y proporciona las fechas deseadas
start_date = '2011-11-05'
end_date = '2013-09-08'
total_users, recommend_percentage = count_reviews_between_dates(df_reviews, start_date, end_date)

print(f'Cantidad de usuarios que realizaron reseñas: {total_users}')
print(f'Porcentaje de recomendación: {recommend_percentage:.2f}%')


Cantidad de usuarios que realizaron reseñas: 2167
Porcentaje de recomendación: 98.16%


Tercera funcion

In [12]:
def genre(género: str):
    # Filtrar df_games para obtener solo los juegos del género especificado
    games_by_genre = df_games[df_games['genres'].str.contains(género, case=False)]
    
    # Unir df_items con games_by_genre para obtener información de playtime
    merged_df = df_items.merge(games_by_genre, left_on='user_id', right_on='id', how='inner')
    
    # Calcular la suma del playtime para cada juego
    playtime_sum = merged_df.groupby('id')['playtime'].sum().reset_index()
    
    # Ordenar el DataFrame por PlayTimeForever en orden descendente
    playtime_sum_sorted = playtime_sum.sort_values(by='playtime', ascending=False)
    
    # Encontrar el índice del juego de género específico en el ranking
    index_of_genre = playtime_sum_sorted[playtime_sum_sorted['id'] == género].index
    
    # Devolver el puesto (indice + 1) si se encuentra, de lo contrario, devolver "No encontrado"
    if not index_of_genre.empty:
        return index_of_genre[0] + 1
    else:
        return "No encontrado"

# Ejemplo de uso para encontrar el puesto del género "Aventura"
puesto = genre("Action")
print(f"El género 'Casual' ocupa el puesto {puesto} en el ranking de PlayTimeForever.")




El género 'Casual' ocupa el puesto No encontrado en el ranking de PlayTimeForever.


In [47]:
import pandas as pd

def genre(género):
    # Convertir el género ingresado en una lista si no lo es
    if not isinstance(género, list):
        género = [género]

    # Unir df_games y df_items en df_merged
    df_merged = df_games.merge(df_items, left_on='id', right_on='item_id')

    # Unir df_merged y df_reviews en df_final
    df_final = df_merged.merge(df_reviews, left_on='id', right_on='item_id')

    # Filtrar las filas que coincidan con los géneros especificados
    df_filtered = df_final[df_final['genres'].apply(lambda x: any(g in x for g in género))]

    # Agrupar por género y sumar el playtime
    df_playtime = df_filtered.groupby('genres')['playtime'].sum().reset_index()

    # Ordenar por playtime descendente
    df_playtime = df_playtime.sort_values('playtime', ascending=False)

    # Encontrar la posición del género
    result = {}
    for g in género:
        row = df_playtime[df_playtime['genres'].str.contains(g, case=False)]
        if not row.empty:
            result[g] = row.index[0] + 1
        else:
            result[g] = 'No encontrado'

    return result

# Ejemplo de uso para encontrar el puesto del género "Action"
puestos = genre(["Action", "Adventure"])
print("Puestos de género:")
for g, puesto in puestos.items():
    print(f"El género '{g}' ocupa el puesto {puesto} en el ranking de PlayTimeForever.")


TypeError: unhashable type: 'list'

In [20]:
puesto = genre('Action') 
print(puesto)

TypeError: unhashable type: 'list'

In [8]:
import pandas as pd

# Importar el archivo 'reviews.pkl' en formato Pickle
df_reviews = pd.read_pickle('reviews.pkl')

# Importar el archivo 'items.pkl' en formato Pickle
df_items = pd.read_pickle('playtime.pkl')

# Importar el archivo 'games.pkl' en formato Pickle
df_games = pd.read_pickle('games.pkl')

# Imprimir información de df_reviews
print("Información de df_reviews:")
print(df_reviews.info())

# Imprimir información de df_items
print("\nInformación de df_items:")
print(df_items.info())

# Imprimir información de df_games
print("\nInformación de df_games:")
print(df_games.info())


Información de df_reviews:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59305 entries, 0 to 59304
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   item_id    59305 non-null  object        
 1   recommend  59305 non-null  int32         
 2   review     59305 non-null  object        
 3   user_id    59305 non-null  object        
 4   posted     59280 non-null  datetime64[ns]
dtypes: datetime64[ns](1), int32(1), object(3)
memory usage: 2.0+ MB
None

Información de df_items:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88310 entries, 0 to 88309
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   user_id   88310 non-null  object 
 1   playtime  88310 non-null  float64
 2   item_id   88310 non-null  object 
dtypes: float64(1), object(2)
memory usage: 2.0+ MB
None

Información de df_games:
<class 'pandas.core.frame.DataFrame'>
Index: 28533

In [ ]:
def users_reviews_between_dates(merged_df, start_date, end_date):
    filtered_df = merge_df[(merged_df['posted'] == start_date) & (merged_df['posted'] == end_date)]
    total_users = filtered_df['user_id'].nunique()
    recommend_percentage = filtered_df['recommend_percentage']
    return total_users, recommend_percentage

In [ ]:
start_date = pd.to_datetime(start_date)
    end_date = pd.to_datetime(end_date)
    
    # Filtrar las reviews entre las fechas dadas
    filtered_reviews = reviews[(reviews['posted'] >= start_date) & (reviews['posted'] <= end_date)]
    
    # Calcular la cantidad de usuarios únicos que realizaron reviews en el período
    unique_users = filtered_reviews['user_id'].nunique()
    
    # Calcular el porcentaje de recomendación
    recommendation_percentage = (filtered_reviews['recommend'].sum() /

In [ ]:
# Lista de columnas relevantes
columnas_similares = ['genres', 'tags', 'specs']

# Función para combinar listas en una cadena
def combinar_listas(row):
    combined = ' '.join(item for sublist in row if sublist for item in sublist)
    return combined

# Crear una nueva columna "variables_juntas" con las listas combinadas en cadenas
df_sg_nec['variables_juntas'] = df_sg_nec[columnas_similares].apply(combinar_listas, axis=1)
# Crear una matriz TF-IDF
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(df_sg_nec['variables_juntas'])

In [ ]:
# Reducción de dimensionalidad con Truncated SVD
num_components = 50  # Número de componentes a mantener
svd = TruncatedSVD(n_components=num_components)
svd_matrix = svd.fit_transform(tfidf_matrix)
# Calcular la similitud de coseno en la matriz reducida
cosine_sim_svd = cosine_similarity(svd_matrix, svd_matrix)

In [ ]:
def sentiment_analysis(año: int):
    # Unir los DataFrames en base a una columna común
    df_merged = pd.merge(steam, reviews, left_on='item_id', right_on='item_id')

    # Convierte 'releasedate' a datetime y extrae el año
    df_merged['year'] = pd.to_datetime(df_merged['releasedate']).dt.year

    # Filtra el DataFrame por el año dado
    df_año = df_merged[df_merged['year'] == año]

    # Cuenta las ocurrencias de cada sentimiento
    conteo_sentimientos = df_año['sentiment_analysis'].valu